# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [7]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from ratelimit import limits
import time
import json

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [8]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

620

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).

> **HINT:** The OpenWeatherMap API only allows 60 calls per minute under their free plan. Try using `time.sleep(60)` after each set of 50 cities to avoid API errors due to large calls.

In [9]:
# Setup API Calling
# url
url = "http://api.openweathermap.org/data/2.5/weather?"
# url = "https://openweathermap.org/api"

# units
unit = "units=Imperial"

# api
api = "&APPID=" + weather_api_key

# response 
response = requests.get( url + unit + api).json()

# print statement
print(response)

# Storage for api call data
city_info = {"City": [], 
             "Lat": [], 
             "Lng": [],
             "Max Temp": [],
             "Humidity": [],
             "Cloudiness": [],
             "Wind Speed": [],
             "Country": [], 
             "Date": []}

# hint, limit api calls
minute = 60
@limits(calls=60, period=minute)

# api call loop
def call_api():
    
    for i, city in enumerate(cities):
        city = requests.get(url + unit + "&q=" + cities[i] + api).json()
        
        try:
            print("Processing city " + str(i) + ": " + city["name"])
        
            city_info["City"].append(city["name"])
            city_info["Lat"].append(city["coord"]["lat"])
            city_info["Lng"].append(city["coord"]["lon"])
            city_info["Max Temp"].append(city["main"]["temp_max"])
            city_info["Humidity"].append(city["main"]["humidity"])
            city_info["Cloudiness"].append(city["clouds"]["all"])
            city_info["Wind Speed"].append(city["wind"]["speed"])
            city_info["Country"].append(city["sys"]["country"])
            city_info["Date"].append(city["dt"])
            
        except:
            print("Incomplete..., skip over missing city info.")
            pass
    print("Complete")
    
call_api()


            
            
        

{'cod': '400', 'message': 'Nothing to geocode'}
Incomplete..., skip over missing city info.
Processing city 1: Lorengau
Processing city 2: Faanui
Processing city 3: Alice Springs
Processing city 4: Puerto Ayora
Processing city 5: Homer
Processing city 6: Saint-Pierre
Processing city 7: Narsaq
Processing city 8: Ushuaia
Processing city 9: Isangel
Processing city 10: Tual
Processing city 11: Bereda
Incomplete..., skip over missing city info.
Processing city 13: Albany
Incomplete..., skip over missing city info.
Incomplete..., skip over missing city info.
Processing city 16: Upernavik
Processing city 17: Hay River
Processing city 18: Clyde River
Processing city 19: Cabo San Lucas
Processing city 20: Awjilah
Processing city 21: Luena
Processing city 22: Ürümqi
Processing city 23: Bethel
Processing city 24: Lebu
Processing city 25: Vilyuysk
Processing city 26: Ponta do Sol
Incomplete..., skip over missing city info.
Processing city 28: Wencheng
Processing city 29: Richards Bay
Processing ci

Processing city 260: Mastic Beach
Processing city 261: Savannah Bight
Processing city 262: Vila Velha
Processing city 263: Caravelas
Processing city 264: Fevik
Processing city 265: Ixtapa
Processing city 266: Codrington
Processing city 267: Huangpi
Processing city 268: Comodoro Rivadavia
Incomplete..., skip over missing city info.
Processing city 270: Omboué
Processing city 271: Juchipila
Processing city 272: Krasnogvardeyskoye
Processing city 273: Badiraguato
Processing city 274: Deogarh
Processing city 275: Uhryniv
Processing city 276: Pevek
Processing city 277: Minsk
Processing city 278: Almaznyy
Processing city 279: Ribeira Grande
Processing city 280: Yar-Sale
Processing city 281: Marawi
Incomplete..., skip over missing city info.
Processing city 283: Baykit
Processing city 284: Staszów
Incomplete..., skip over missing city info.
Processing city 286: Saint-Philippe
Processing city 287: Auki
Processing city 288: Udachny
Incomplete..., skip over missing city info.
Processing city 290

Processing city 523: Arinos
Processing city 524: Grand Gaube
Processing city 525: Baghdad
Processing city 526: Alushta
Processing city 527: Tarauacá
Processing city 528: Verkhnevilyuysk
Processing city 529: Tepalcatepec
Processing city 530: Pisco
Processing city 531: Ýpsonas
Processing city 532: Tokonou
Processing city 533: Smolenka
Processing city 534: Dossor
Processing city 535: Kovdor
Processing city 536: Whitianga
Processing city 537: Tuatapere
Processing city 538: Tegul'det
Processing city 539: Belaya Gora
Processing city 540: Saint-Louis
Incomplete..., skip over missing city info.
Processing city 542: Kinablangan
Processing city 543: Shaunavon
Processing city 544: High Rock
Processing city 545: Pilar
Processing city 546: Medvedka
Incomplete..., skip over missing city info.
Processing city 548: Bandarbeyla
Incomplete..., skip over missing city info.
Processing city 550: Laguna
Incomplete..., skip over missing city info.
Processing city 552: ‘Āhuimanu
Processing city 553: High Leve

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [10]:
# Creating DataFrame
city_info_df = pd.DataFrame(city_info)

# Export as CSV file
city_info_df.to_csv(output_data_file, index_label="City_ID")

FileNotFoundError: [Errno 2] No such file or directory: 'output_data/cities.csv'

In [ ]:
# Inspect DataFrame
city_info_df

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression